# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import gmaps
from scipy.stats import linregress
import scipy.stats as st

# gmaps API Keys
from config import (api_key)
#from config import (gkey)

In [2]:
# pip install citipy
from citipy import citipy

## Generate Cities List

In [3]:
cities_list = []
latitude_longitude_pairs = []
np.random.seed(250)  # to allow others to duplicate 
lats = np.random.randint(-90.00, 90.00, size=2000)
longs = np.random.randint(-180.00, 180.00, size=2000)
latitude_longitude_pairs = zip(lats, longs) 

city_table_df = pd.DataFrame(columns = ['Latitude', 'Longitude', 'City'])

for latitude_longitude_pair in latitude_longitude_pairs:
    returned_city = citipy.nearest_city(latitude_longitude_pair[0],latitude_longitude_pair[1]).city_name
    
    if returned_city not in cities_list:
       cities_list.append(returned_city) 
       city_table_df = city_table_df.append({'Latitude' : latitude_longitude_pair[0], 'Longitude' : latitude_longitude_pair[1], 'City' : returned_city}, ignore_index = True)  
        
#len(list_of_cities)  
city_table_df.head(20)

,Latitude,Longitude,City
0,20,-54,saint-francois
1,-13,156,gizo
2,-65,-173,vaini
3,-32,79,bambous virieux
4,-39,-143,mataura
5,66,-67,pangnirtung
6,-61,22,bredasdorp
7,-1,104,simpang
8,75,-11,husavik
9,22,-105,ruiz


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

query_url = f"{url}appid={api_key}&units={units}&q="

In [5]:
# set up lists to hold reponse info
import pprint

lat = []
long = []
maxtemp = []
humid = []
wind = []
speed = []
cloud = []
cities2 = []
country = []
dt = []

# Loop through the list of cities and perform a request for data on each
for city in cities_list:
    response = requests.get(query_url + city).json()
        
    try:
        #pprint.pprint(response)
        lat.append(response['coord']['lat'])
        long.append(response['coord']['lon'])
        maxtemp.append(response['main']['temp_max'])
        humid.append(response['main']['humidity'])
        wind.append(response['wind']['speed'])
        cloud.append(response['clouds']['all'])
        cities2.append(response['name'])
        country.append(response['sys']['country'])
        dt.append(response['dt'])
   
    except:
          pass

print('this message') 

this message


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [6]:
#print(cities2)
weather_datasofar = {"city": cities2,
                     "country": country,
                     "latitude": lat,
                     "longitude": long,
                     "max temp": maxtemp,
                     "humidity": humid,
                     "wind speed": wind,
                     "cloudiness": cloud,
                     "dt": dt}
weather_guide_df = pd.DataFrame(weather_datasofar)
weather_guide_df 

,city,country,latitude,longitude,max temp,humidity,wind speed,cloudiness,dt
0,Saint-François,FR,46.4154,3.9054,44.60,93,3.44,90,1619922939
1,Gizo Government Station,SB,-8.1030,156.8419,82.53,71,1.74,100,1619922940
2,Vaini,TO,-21.2000,-175.2000,78.80,73,17.27,75,1619922725
3,Bambous Virieux,MU,-20.3428,57.7575,73.99,94,6.91,40,1619922762
4,Mataura,NZ,-46.1927,168.8643,59.00,80,7.00,72,1619922716
...,...,...,...,...,...,...,...,...,...
666,Kafanchan,NG,9.5833,8.3000,74.34,70,4.45,15,1619923124
667,Bhasāwar,IN,27.0333,77.0333,92.01,20,5.26,100,1619923124
668,Znamenskoye,RU,57.1281,73.8282,50.49,42,12.77,91,1619923125
669,Atbasar,KZ,51.8000,68.3333,53.19,36,14.14,38,1619923125


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [7]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [8]:
from config import (api_key, gkey)

gmaps.configure(api_key=gkey) 
#weather_guide.dtypes

In [9]:
locations = weather_guide_df[["latitude", "longitude"]].astype(float)

In [10]:
fig = gmaps.figure()

humidity_rate = weather_guide_df["humidity"].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
humidity_numbers = weather_guide_df["humidity"].tolist()
humidity_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0 , 150, 0.4)',
                                    scale=2, info_box_content=[f"Humidity: {humidity_number}" for humidity_number in 
                                                               humidity_numbers])

fig = gmaps.figure()
fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
#mouse_metadata_df.drop(mouse_metadata_df[mouse_metadata_df['Mouse ID']=='g989'].index, inplace=True)
weather_guide_df.drop(weather_guide_df[weather_guide_df['cloudiness']>0].index, inplace=True)
weather_guide_df.drop(weather_guide_df[weather_guide_df['wind speed']>10].index, inplace=True)
weather_guide_df.drop(weather_guide_df[weather_guide_df['max temp']<70].index, inplace=True)
weather_guide_df.drop(weather_guide_df[weather_guide_df['max temp']>80].index, inplace=True)

#len(weather_guide_df)
weather_guide_df.head(20)

,city,country,latitude,longitude,max temp,humidity,wind speed,cloudiness,dt
105,Saint-Philippe,RE,-21.3585,55.7679,73.40,68,5.75,0,1619922967
113,Sakakah,SA,29.9697,40.2064,73.40,16,3.44,0,1619922720
202,Charters Towers,AU,-20.1000,146.2667,79.93,46,9.64,0,1619922992
321,Saint-Joseph,RE,-21.3667,55.6167,73.40,68,5.75,0,1619923025
328,Iralaya,HN,15.0000,-83.2333,79.83,75,9.82,0,1619923027
428,Naze,JP,28.3667,129.4833,73.40,43,9.22,0,1619923056
551,Riyadh,SA,24.6877,46.7219,77.00,19,3.89,0,1619923026
638,Wembley,AU,-31.9333,115.8000,70.00,45,1.14,0,1619923116


In [15]:
# base url

hotel_name = []

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in weather_guide_df.iterrows():
    
    target_coordinate_1 = row['latitude']
    target_coordinate_2 = row['longitude']
    target_coordinates = str(target_coordinate_1) + ',' + str(target_coordinate_2)

    params = {
    "location": target_coordinates,
    "radius": 5000,
    "types": "lodging",
    "key": gkey
         }
    #print (params)
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        hotel_name.append(results[0]['name'])
    except:
        hotel_name.append('No Lodging Located')
        
weather_guide_df['Hotel Name'] = hotel_name
weather_guide_df.head(20)

,city,country,latitude,longitude,max temp,humidity,wind speed,cloudiness,dt,Hotel Name
105,Saint-Philippe,RE,-21.3585,55.7679,73.40,68,5.75,0,1619922967,"Chambres d'hôte ""La Trinité"""
113,Sakakah,SA,29.9697,40.2064,73.40,16,3.44,0,1619922720,Fakhamat Al Orjoana Apartment
202,Charters Towers,AU,-20.1000,146.2667,79.93,46,9.64,0,1619922992,Cattleman's Rest Motor Inn
321,Saint-Joseph,RE,-21.3667,55.6167,73.40,68,5.75,0,1619923025,"""Plantation Bed and Breakfast"
328,Iralaya,HN,15.0000,-83.2333,79.83,75,9.82,0,1619923027,Iglesia morava renovada
428,Naze,JP,28.3667,129.4833,73.40,43,9.22,0,1619923056,奄美ポートタワーホテル
551,Riyadh,SA,24.6877,46.7219,77.00,19,3.89,0,1619923026,"InterContinental Riyadh, an IHG Hotel"
638,Wembley,AU,-31.9333,115.8000,70.00,45,1.14,0,1619923116,Quest on Rheola


In [16]:
locations = weather_guide_df[["latitude", "longitude"]].astype(float)

hotel_names = weather_guide_df["Hotel Name"].tolist()
city_names = weather_guide_df["city"].tolist()
country_codes = weather_guide_df["country"].tolist()

info_box_content = [
  f"Hotel: {hotel_name}, City: {city_name}, Country: {country_code}"
  for hotel_name, city_name, country_code
  in zip(hotel_names, city_names, country_codes)
]

hotel_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0 , 150, 0.4)',
                                    scale=2)

fig = gmaps.figure()

markers=gmaps.marker_layer(locations, info_box_content)
fig.add_layer(markers)                                 
fig.add_layer(heat_layer)                           
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))